## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Medical LLM - 14B](https://aws.amazon.com/marketplace/pp/prodview-u5vx4onx5kucy)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

- **Model**: `JSL-Medical-LLM-14B`  
- **Model Description**: Medical LLM - 14B

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [9]:
import os
import re
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [10]:
model_name = "JSL-Medical-LLM-14B"

real_time_inference_instance_type = "ml.g5.12xlarge"
batch_transform_inference_instance_type = "ml.g5.12xlarge"

## Model Configuration Documentation  

### Default Configuration  
The container comes with the following default configurations:  

| Parameter                  | Default Value | Description                                                                   |  
|----------------------------|---------------|-------------------------------------------------------------------------------|  
| **`dtype`**                | `auto`        | Data type for model weights and activations (automatically determined)        |  
| **`tensor_parallel_size`** | Auto          | Automatically set to the number of available GPUs (`torch.cuda.device_count()`)|  
| **`host`**                 | `0.0.0.0`     | Host name                                                                     |  
| **`port`**                 | `8080`        | Port number                                                                   |  
| **`tokenizer_mode`**       | `auto`        | Tokenizer mode (automatically determined)                                     |  
| **`reasoning_parser`**     | `qwen3`       | Reasoning parser to use for extracting reasoning content from the model output|  

### Hardcoded Settings  
The following settings are hardcoded in the container and cannot be changed:  

| Parameter       | Value           | Description                           |  
|-----------------|-----------------|---------------------------------------|  
| **`model`**     | `/opt/ml/model` | Model path where SageMaker mounts the model |  

### Configurable Environment Variables  
You can customize the vLLM server by setting environment variables when creating the model.  

**Any parameter from the [vLLM documentation](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#vllm-serve) can be set using the corresponding environment variable with the `SM_VLLM_` prefix.**  

The container uses a script similar to the [SageMaker entrypoint example](https://docs.vllm.ai/en/v0.8.5/getting_started/examples/sagemaker-entrypoint.html) from the vLLM documentation to convert environment variables to command-line arguments.  

---  

## Input Format  

### 1. Chat Completion  

#### Example Payload  
```json  
{  
    "model": "/opt/ml/model",  
    "messages": [  
        {"role": "system", "content": "You are a helpful medical assistant."},  
        {"role": "user", "content": "What should I do if I have a fever and body aches?"}  
    ],  
    "max_tokens": 1024,  
    "temperature": 0.7  
}  
```  

For additional parameters:  
- [ChatCompletionRequest](https://github.com/vllm-project/vllm/blob/v0.9.2/vllm/entrypoints/openai/protocol.py#L223)  
- [OpenAI's Chat API](https://platform.openai.com/docs/api-reference/chat/create)  

---  

### 2. Text Completion  

#### Single Prompt Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": "How can I maintain good kidney health?",  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

#### Multiple Prompts Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": [  
        "How can I maintain good kidney health?",  
        "What are the best practices for kidney care?"  
    ],  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

Reference:  
- [CompletionRequest](https://github.com/vllm-project/vllm/blob/v0.9.2/vllm/entrypoints/openai/protocol.py#L741)  
- [OpenAI's Completions API](https://platform.openai.com/docs/api-reference/completions/create)  

---  

### Important Notes:
- **Streaming Responses:** Add `"stream": true` to your request payload to enable streaming
- **Model Path Requirement:** Always set `"model": "/opt/ml/model"` (SageMaker's fixed model location)

### Initial setup

In [11]:
prompt1 = """A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.

Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""

prompt2 = "What should I do if I have a fever and body aches?"

prompts = [
    "How can I maintain good kidney health?",
    "What are the symptoms of high blood pressure?"
]



In [12]:
system_prompt ="You are a helpful medical assistant. Provide accurate, evidence-based information in response to the following question. Organize the response with clear hierarchical headings and include a conclusion if necessary."

## 2. Create a deployable model from the model package.

In [13]:
model = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

## 3. Create SageMaker Endpoint

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [15]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

---------------!

### 3.1 Real-time inference via Amazon SageMaker Endpoint

In [16]:
def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

In [17]:
def print_colored(text, color='green'):
    colors = {
        'green': '\033[92m',
        'reset': '\033[0m',
    }
    color_code = colors.get(color, colors['reset'])
    print(f"{color_code}{text}{colors['reset']}", end="", flush=True)

#### Chat Completion

In [27]:
input_data = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1},
    ],
    "max_tokens": 8192,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 20,
    "min_p": 0.0,
    "presence_penalty":0.0
}

result = invoke_realtime_endpoint(input_data)
message = result["choices"][0]["message"]

reasoning_content = message.get("reasoning_content")
content = message.get("content")

print_colored(reasoning_content, 'green')
print_colored(content, 'reset')


Okay, so I need to figure out the best treatment for this 23-year-old pregnant woman with burning upon urination at 22 weeks gestation. Let me start by recalling the information given. She has symptoms of dysuria for one day, which has worsened despite increased water intake and cranberry extract. She's otherwise feeling well, and her vital signs are normal except for a slightly low temperature (but within normal range). The physical exam shows no costovertebral angle tenderness, and her uterus is gravid.

First, the primary concern here is a urinary tract infection (UTI) given the symptoms of dysuria. Since she's pregnant, I need to be cautious about which antibiotics are safe during pregnancy. The options are Ampicillin, Ceftriaxone, Ciprofloxacin, Doxycycline, and Nitrofurantoin.

I remember that during pregnancy, certain antibiotics are preferred because others can harm the fetus. For example, fluoroquinolones like Ciprofloxacin are generally avoided in pregnancy due to potential 

#### Text Completion

In [29]:
input_data ={
        "model": "/opt/ml/model",
        "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
        "max_tokens": 2048,
        "temperature": 0.7,
        "top_p": 0.95,
        "top_k": 20,
        "min_p": 0.0,
        "presence_penalty":0.0
    }

result = invoke_realtime_endpoint(input_data)
output_text = result['choices'][0]['text']
print(output_text)

 # Managing Fever and Body Aches

If you have a fever and body aches, it's important to take appropriate steps to manage your symptoms and determine if medical attention is needed. Here's a structured approach:

## Initial Self-Care

- **Rest**: Allow your body time to recover by getting plenty of sleep and avoiding strenuous activities
- **Hydration**: Drink plenty of fluids to prevent dehydration
- **Cooling measures**: Use cool compresses or take lukewarm baths to help reduce fever
- **Over-the-counter medications**: Consider acetaminophen (Tylenol) or ibuprofen (Advil, Motrin) for fever and pain relief, following package instructions

## When to Seek Medical Attention

- **High fever**: If your temperature reaches 103°F (39.4°C) or higher
- **Persistent symptoms**: If symptoms last more than 3-4 days or worsen over time
- **Severe symptoms**: Difficulty breathing, chest pain, severe headache, confusion, or severe dehydration
- **Underlying conditions**: If you have a chronic illnes

### 3.2 Real-time inference response as a stream via Amazon SageMaker Endpoint

In [32]:
def invoke_streaming_endpoint(record):
    try:
        response = sm_runtime.invoke_endpoint_with_response_stream(
            EndpointName=model_name,
            Body=json.dumps(record),
            ContentType="application/json",
            Accept="text/event-stream"
        )

        is_chat_completion = "messages" in record

        for event in response["Body"]:
            if "PayloadPart" in event:
                chunk = event["PayloadPart"]["Bytes"].decode("utf-8")
                if chunk.startswith("data:"):
                    try:
                        data = json.loads(chunk[5:].strip())

                        if "choices" not in data or len(data["choices"]) == 0:
                            continue

                        choice = data["choices"][0]
                        if is_chat_completion:
                            if "delta" in choice:
                                delta = choice["delta"]

                                if "reasoning_content" in delta:
                                    yield {'type': 'reasoning', 'content': delta["reasoning_content"]}

                                elif "content" in delta:
                                    yield {'type': 'content', 'content': delta["content"]}
                        else:

                            if "text" in choice:
                                yield {'type': 'text', 'content': choice["text"]}

                    except json.JSONDecodeError:
                        continue
                        
            elif "ModelStreamError" in event:
                error = event["ModelStreamError"]
                yield {'type': 'error', 'content': f"\nStream error: {error['Message']} (Error code: {error['ErrorCode']})"}
                break
            elif "InternalStreamFailure" in event:
                failure = event["InternalStreamFailure"]
                yield {'type': 'error', 'content': f"\nInternal stream failure: {failure['Message']}"}
                break
                
    except Exception as e:
        yield {'type': 'error', 'content': f"\nAn error occurred during streaming: {str(e)}"}

In [33]:
def handle_streaming_chunk(chunk):
    """
    Handles and prints a streaming chunk with appropriate formatting
    Args:
        chunk: Dictionary with 'type' and 'content' keys
    """
    if chunk['type'] == 'reasoning':
        print_colored(chunk['content'], 'green')
    elif chunk['type'] == 'content':
        print_colored(chunk['content'], 'reset')
    elif chunk['type'] == 'text':  # For text completion
        print(chunk['content'], end="", flush=True)
    elif chunk['type'] == 'error':
        print_colored(chunk['content'], 'reset')

#### Chat Completion

In [36]:
payload = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt },
        {"role": "user", "content": prompt1},
    ],
    "max_tokens": 8192,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 20,
    "min_p": 0.0,
    "presence_penalty": 0.0,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    handle_streaming_chunk(chunk)


Okay, let's see. The patient is a 23-year-old woman at 22 weeks gestation with burning upon urination that started a day ago and has worsened despite drinking more water and taking cranberry extract. Her temperature is normal, blood pressure is okay, pulse is normal, respirations are normal, oxygen saturation is good. Physical exam shows no costovertebral angle tenderness and a gravid uterus. So, she's probably presenting with a urinary tract infection (UTI), likely a lower UTI since she has burning on urination.

Now, the question is about the best treatment. The options are antibiotics: Ampicillin, Ceftriaxone, Ciprofloxacin, Doxycycline, Nitrofurantoin.

First, I need to consider the safety of each antibiotic during pregnancy. The third trimester is when certain antibiotics might be contraindicated, but she's in the second trimester here. Let me recall the guidelines for UTI management in.

In pregnancy, the treatment of UTIs involves antibiotics that are safe for the fetus. Common

#### Text Completion

In [37]:
payload = {
    "model": "/opt/ml/model",
    "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
    "max_tokens": 2048,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 20,
    "min_p": 0.0,
    "presence_penalty":0.0,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    handle_streaming_chunk(chunk)

 # Fever and Body Aches: Self-Care and When to Seek Medical Help

## Initial Self-Care Measures

1. **Stay hydrated**:
   - Drink plenty of fluids (water, clear broths, electrolyte solutions)
   - Avoid alcohol and caffeine

2. **Rest**:
   - Get adequate sleep and reduce physical activity
   - Consider taking short naps if needed

3. **Fever**:
   - Take over-the-counter fever-reducing medications like acetaminophen or ibuprofen (follow package instructions)
   - Use lukewarm (not cold) compresses on forehead or neck
   - Consider a cool bath or shower (not cold)

4. **Comfort measures**:
   - Wear loose, breathable clothing
   - Keep room temperature moderate (between 68-72°F/20-22°C)
   - Use a humidifier if air is dry

## When to Seek Medical Attention

Contact a provider or emergency services if you:
- Fever above 103°F (39.4°C) in adults or 100.4°F (38°C) in children
- Persistent fever lasting more than 3-5 days
- Difficulty breathing or shortness of breath
- Chest pain or pressu

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [38]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 5. Batch inference

In [21]:
validation_json_file_name1 = "input1.json"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/"


def write_and_upload_to_s3(input_data, file_name):
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_name}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [22]:
input_json_data1 = json.dumps(
    {
        "model": "/opt/ml/model",
        "prompt": [f"{system_prompt}\n\nUser: {prompt}\n\nAssistant:" for prompt in prompts],
        "max_tokens": 2048,
        "temperature": 0.6,
        "top_p": 0.95,
        "top_k": 20,
        "min_p": 0.0,
        "presence_penalty":0.0,
    }
)

write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
)
transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)
    result = json.loads(response["Body"].read().decode("utf-8"))
    
    for idx, choice in enumerate(result.get("choices", [])):
        print(f"Response {idx + 1}:\n{choice.get('text', '')}\n{'=' * 75}")


In [26]:
retrieve_json_output_from_s3(validation_json_file_name1)

Response 1:
 # Maintaining Good Kidney Health

## General Lifestyle Recommendations

### Hydration
- Drink adequate water daily (8-10 glasses for most adults)
- Monitor urine color (pale yellow is ideal)
- Limit sugary drinks and excessive caffeine

### Diet
- Follow a balanced diet rich in fruits, vegetables, whole grains, and lean proteins
- Limit sodium intake to less than 2,300 mg daily
- Reduce processed foods and red meat consumption
- Control protein intake, especially if you have kidney disease

### Physical Activity
- Engage in regular exercise (at least 150 minutes of moderate activity weekly)
- Maintain a healthy weight
- Avoid smoking and excessive alcohol consumption

## Medical Management

### Regular Checkups
- Get routine kidney function tests (especially if you have risk factors)
- Monitor blood pressure regularly
- Follow treatment plans for diabetes, hypertension, and other conditions

### Medication Management
- Use medications as prescribed, especially for blood pr

Congratulations! You just verified that the batch transform job is working as expected. Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

In [27]:
model.delete_model()

INFO:sagemaker:Deleting model with name: JSL-Medical-LLM-14B-2025-07-11-09-18-49-255


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

